# 💾 Notebook: Persistencia de Datos Transformados y Parámetros

### **Paso 1: Ejecutar Transformaciones y Obtener Parámetros de Reversión**

Volvemos a ejecutar la lógica de las transformaciones, pero con un enfoque en capturar los valores clave necesarios para la reversión de la serie.

In [1]:
import pandas as pd
import numpy as np
import json # Necesario para guardar los parámetros en formato JSON
from scipy.stats import boxcox

# --- 1. CARGA Y PREPARACIÓN DE DATOS ---
try:
    df = pd.read_csv('pinar_del_rio_temperaturas_LIMPIO_FINAL.csv')
except FileNotFoundError:
    df = pd.read_csv('dataset/pinar_del_rio_temperaturas_LIMPIO_FINAL.csv')

df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.set_index('Fecha')


# --- 2. BOX-COX (Obtener Lambda y Transformar) ---
data_max = df['Temp_Maxima_C'].values
_, lambda_max = boxcox(data_max) 
df['BoxCox_Max'] = boxcox(df['Temp_Maxima_C'], lmbda=lambda_max)

data_min = df['Temp_Minima_C'].values
_, lambda_min = boxcox(data_min)
df['BoxCox_Min'] = boxcox(df['Temp_Minima_C'], lmbda=lambda_min)

print(f"Lambdas Box-Cox obtenidos: Max={lambda_max:.4f}, Min={lambda_min:.4f}")


# --- 3. DIFERENCIACIÓN (d=1) ---
df['Diff_BoxCox_Max'] = df['BoxCox_Max'].diff(1)
df['Diff_BoxCox_Min'] = df['BoxCox_Min'].diff(1)


# --- 4. DATASETS FINALES ---
# 4a. DataFrame para el Modelado (Estacionario)
df_stationary = df[['Diff_BoxCox_Max', 'Diff_BoxCox_Min']].dropna()
print(f"DataFrame Estacionario listo. Tamaño: {len(df_stationary)} filas.")


# 4b. Obtener el último valor Box-Cox ANTES de la diferenciación.
# Este valor es el "seed" o "semilla" que se necesita para revertir la diferenciación.
last_boxcox_max = df['BoxCox_Max'].iloc[-1]
last_boxcox_min = df['BoxCox_Min'].iloc[-1]

# También capturamos los últimos valores originales, aunque la reversión se hace sobre Box-Cox,
# es útil tener el último valor original para referencia y validación.
last_original_max = df['Temp_Maxima_C'].iloc[-1]
last_original_min = df['Temp_Minima_C'].iloc[-1]
last_original_date = df.index[-1].strftime('%Y-%m-%d')

print(f"Última fecha de observación: {last_original_date}")

Lambdas Box-Cox obtenidos: Max=4.3777, Min=4.2651
DataFrame Estacionario listo. Tamaño: 670 filas.
Última fecha de observación: 2025-11-01


### **Paso 2: Guardar Datos Transformados (CSV)**

Guardamos el DataFrame que el modelo utilizará para el entrenamiento.

In [2]:
# Guardar el dataset estacionario en un nuevo archivo CSV
df_stationary.to_csv('dataset/datos_temperaturas_estacionarios.csv', index=True)

print("\n=========================================================================")
print("✅ Archivo 'datos_temperaturas_estacionarios.csv' guardado exitosamente.")
print("=========================================================================")


✅ Archivo 'datos_temperaturas_estacionarios.csv' guardado exitosamente.


### **Paso 3: Guardar Parámetros de Reversión (JSON)**

Guardamos todos los coeficientes y valores iniciales necesarios para la función de De-Transformación de la predicción.

In [3]:
# Estructura del diccionario de parámetros
transformation_params = {
    "ultima_fecha_historica": last_original_date,
    
    # 1. Parámetros de Box-Cox (Para revertir Box-Cox)
    "boxcox_lambda": {
        "maxima": lambda_max,
        "minima": lambda_min
    },
    
    # 2. Último valor Box-Cox (Para revertir la Diferenciación)
    "ultimo_valor_boxcox": {
        # Este es el valor de Y(t-1) Box-Cox que se usará como semilla para la reversión
        "maxima": last_boxcox_max,
        "minima": last_boxcox_min
    },
    
    # 3. Último valor Original (Solo para referencia y validación)
    "ultimo_valor_original": {
        "maxima": last_original_max,
        "minima": last_original_min
    }
}

# Guardar en archivo JSON
with open('dataset/parametros_reversion.json', 'w') as f:
    json.dump(transformation_params, f, indent=4)

print("✅ Archivo 'parametros_reversion.json' guardado exitosamente.")

✅ Archivo 'parametros_reversion.json' guardado exitosamente.


### **Próximo Paso: Entrenamiento del Modelo**
Ahora tenemos dos activos clave:

datos_temperaturas_estacionarios.csv: El input perfecto para entrenar el modelo.

parametros_reversion.json: La "llave" para interpretar las predicciones en el futuro.

El camino lógico es ahora: Selección del Modelo y Entrenamiento SARIMAX Vectorial.